In [1]:
from turtle import forward
import torch
import torch.nn as nn

# from .modules import Encoder, LayerNorm
from modules import Encoder, LayerNorm

In [33]:
input_items = [[0, 0, 1,2,3,4,5,6], [0, 0, 0, 0,3,4,5,6]]
input_ratings = [[0, 0, .1, .2, .4, .2, .1, .8], [0, 0, 0, 0, .4, .2, .1, .8]]

input_items = torch.tensor(input_items)
input_ratings = torch.tensor(input_ratings)

In [34]:
attention_mask = (input_items > 0).long()
extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # torch.int64

print(attention_mask)
print(extended_attention_mask)

tensor([[0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1]])
tensor([[[[0, 0, 1, 1, 1, 1, 1, 1]]],


        [[[0, 0, 0, 0, 1, 1, 1, 1]]]])


In [35]:
weighted_mask = input_ratings
extended_weighted_mask = weighted_mask.unsqueeze(1).unsqueeze(2)

print(weighted_mask)
print(extended_weighted_mask)

tensor([[0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.1000, 0.8000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.4000, 0.2000, 0.1000, 0.8000]])
tensor([[[[0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.1000, 0.8000]]],


        [[[0.0000, 0.0000, 0.0000, 0.0000, 0.4000, 0.2000, 0.1000, 0.8000]]]])


In [36]:
max_len = attention_mask.size(-1)
attn_shape = (1, max_len, max_len)

print(max_len)
print(attn_shape)

8
(1, 8, 8)


In [38]:
subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1)
subsequent_mask = (subsequent_mask == 0).unsqueeze(1)
subsequent_mask = subsequent_mask.long()

print(subsequent_mask)
print("------------------------------------------------------------------------------")

extended_attention_mask = extended_attention_mask * subsequent_mask 

print(extended_attention_mask)
print("------------------------------------------------------------------------------")

extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

print(extended_attention_mask)
print("------------------------------------------------------------------------------")




tensor([[[[1, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1, 1]]]])
------------------------------------------------------------------------------
tensor([[[[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 1, 0, 0, 0, 0, 0],
          [0, 0, 1, 1, 0, 0, 0, 0],
          [0, 0, 1, 1, 1, 0, 0, 0],
          [0, 0, 1, 1, 1, 1, 0, 0],
          [0, 0, 1, 1, 1, 1, 1, 0],
          [0, 0, 1, 1, 1, 1, 1, 1]]],


        [[[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 0, 0, 1, 1, 0, 0],
          [0, 0, 0, 0, 1, 1, 1, 0],
          [0, 0, 0, 0, 1, 1, 1, 1]]]])
-----------------------------------------------

In [39]:
weighted_mask_by_score = extended_weighted_mask * subsequent_mask

print(weighted_mask_by_score)

# 이 부분 과연 해당 Rating으로 weight를 곱해주는 것이 올바른 선택일까? 

# Test 해볼 수 있는 여지는 2가지가 있다
####################################################################################
#  - 1 ) 학습은 implict과 비슷한 구조를 채택하고 오로지 loss 값을 구할 때만, Explicit 정보를 활용한다. 
    # 이렇게 학습하면, 학습시에 Explicit한 정답은 오로지 외부에서만 참조가 되므로, 
    # 추론시에는 오로지 긍, 부정으로 선택한 결과에 대해 긍정만 시퀀스로 입력해줄 수 있는 방법이 있다. 

####################################################################################
# - 2 ) masking이 0
# masking을 0으로 처리한 것에 대해서 생각해보자
# 그렇게 나온 값에 대해서 학습에 사용하지 않는다는 것 = 해당 위치에 있는 아이템에 대해서 참조하지 않겠다는 것이다. 
# 유저의 점수가 0에서 1사이 일때, 이 값을 관심도라고 하고 해당 위치 아이템의 선호도를 통해서 어텐션 스코어에 유저의 관심도의 역할로 한 번 더 처리해주는 역할을 하게 된다. 

# [] 여기서는 일단 2번 수행 방법을 통해서 접근하게 된다. 
# 확실한 것은, 스코어를 유저의 관심도로 표현함에 따른 리스크
# 점수에 대한 유저의 평가를 각각 표준화 시켜야한다는 것이다. ####### -> 이런 가정 조건이 없다면, 유저가 평가한 점수가, 각각 지멋대로이기 때문에, 균일한 학습이 잘 되지 않을 수 있다. 
# 따라서 해당 부분은 유저별로 다르게 놔두는것이 아니라 같은 분포를 따르도록 만드는 것이 중요할 것으로 판단된다. 
# 왜냐면 해당 모델은 유저가 구분되지 않는 시퀀셜한 정보만으로 해당 시퀀스의 성격을 판단하게 되므로, 
# 유저별 기록의 편차를 줄여주어야 모델이 조금 더 쉽게 학습시킬 수 있을 것으로 보인다.

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.1000, 0.0000],
          [0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.1000, 0.8000]]],


        [[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.4000, 0.0000, 0.0000, 0.0000],
        

In [44]:
weighted_mask_by_score.size()

torch.tensor([[0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1]]) * weighted_mask_by_score

RuntimeError: The size of tensor a (2) must match the size of tensor b (8) at non-singleton dimension 2

In [19]:
subsequent_mask = torch.tril(torch.ones(attn_shape), diagonal=1)
subsequent_mask = (subsequent_mask == 0).unsqueeze(1)
subsequent_mask = subsequent_mask.long()
print(subsequent_mask)

tensor([[[[0, 0, 1, 1, 1, 1, 1, 1, 1],
          [0, 0, 0, 1, 1, 1, 1, 1, 1],
          [0, 0, 0, 0, 1, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 1, 1, 1, 1],
          [0, 0, 0, 0, 0, 0, 1, 1, 1],
          [0, 0, 0, 0, 0, 0, 0, 1, 1],
          [0, 0, 0, 0, 0, 0, 0, 0, 1],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0]]]])
